In [ ]:
import joblib
import pytesseract
import cv2
import matplotlib.pyplot as plt
import pickle
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import joblib
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import os

In [ ]:
import streamlit as st

In [ ]:
mj=joblib.load('/Users/rahul/Downloads/IAM_Words/act_model__joblib_10_epochs')
cnn=joblib.load('/Users/rahul/Downloads/IAM_Words/DTvsHW_Model')

In [ ]:
char_set={'M', "'", '/', ')', 'm', 'q', 'k', 'N', '!', ',', 'o', 'W', '+', '#', '0', 'y', 'd', 'j', 'h', 'I', '-', 'Z', 'F', 'Y', 'g', 'P', 'V', 'e', 'z', 'R', '5', 'p', '1', '?', 'x', 'c', 'D', '.', 'C', 'K', 'w', 'v', 'G', '(', 'T', '3', 'S', ':', 'i', 'A', 'E', 'a', 'U', '6', 'b', 't', '*', '9', '8', 'l', '4', ';', 'X', '2', 's', '7', 'r', 'J', 'O', 'Q', 'B', 'n', 'f', '&', 'u', '"', 'L', 'H'}
char_list = sorted(list(char_set))

In [ ]:
def process_image(img):
    """
    Converts image to shape (32, 128, 1) & normalize
    """
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
    w, h = img.shape

    # Aspect Ratio Calculation
    new_w = 32
    new_h = int(h * (new_w / w))
    img = cv2.resize(img, (new_h, new_w))
    w, h = img.shape

    img = img.astype('float32')

    # Converts each to (32, 128, 1)
    if w < 32:
        add_zeros = np.full((32-w, h), 255)
        img = np.concatenate((img, add_zeros))
        w, h = img.shape

    if h < 128:
        add_zeros = np.full((w, 128-h), 255)
        img = np.concatenate((img, add_zeros), axis=1)
        w, h = img.shape

    if h > 128 or w > 32:
        dim = (128,32)
        img = cv2.resize(img, dim)

    img = cv2.subtract(255, img)
    img = np.expand_dims(img, axis=2)

    # Normalize
    img = img / 255
    return img

In [ ]:
st.title("hand written text prediction")

In [ ]:
uploaded_file = st.file_uploader("Choose an image...", type="jpg")

In [ ]:
from PIL import Image

In [ ]:
def load_image_into_cv2(uploaded_file):
    # Convert the uploaded file to an OpenCV image
    image = Image.open(uploaded_file)
    image_array = np.array(image)
    image_cv = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)
    return image_cv

In [ ]:
def first1(predictable_img):
    predictable_array_images=[]
    predictable_img2 = process_image(predictable_img)
    predictable_array_images.append(predictable_img2)
    print(predictable_array_images[0].shape)
    bimbon_images = np.array(predictable_array_images)
    bimbon_images = tf.expand_dims(bimbon_images, axis=0)
    print(bimbon_images[0].shape)
    # predict outputs on validation images
    prediction = mj.predict(bimbon_images[0])

    print(prediction)

    # use CTC decoder
    decoded = K.ctc_decode(prediction,   
                           input_length=np.ones(prediction.shape[0]) * prediction.shape[1],
                           greedy=True)[0][0]

    out = K.get_value(decoded)
    print('out is ',out)

    for i, x in enumerate(out):
        print("predicted text = ", end = '')
        S=""
        for p in x:
            if int(p) != -1:
                #print(char_list[int(p)], end = '')
                S+=char_list[int(p)]
        new_shape = (32, 128)  # Example new shape
        reshaped_tensor = tf.reshape(bimbon_images[0], new_shape)
        plt.imshow(reshaped_tensor,cmap=plt.cm.gray)
        plt.show()
        return S
        print('\n')

In [ ]:
def HWContours(cnts):
    x=image.shape[1]
    print('x:-',x)
    y=image.shape[0]
    print('y:-',y)
    width_of_image=x
    height_of_image=y
    print(0.5*width_of_image)
    import numpy as np
    a=int(0)
    no_of_c=0
    list_strings=[]
    results  = []
    print('no.of contours are ',len(cnts))
    aspect_ratios=[]
    for c in cnts:
        x, y, w, h = cv2.boundingRect(c)
        if h > 1 and w > 1:
            roi=image[y:y+h,x:x+w]
            predictable_img = cv2.resize(roi, (300,128))
            a+=1

            # Resize the image to the required size (64, 64)
            test_image = cv2.resize(predictable_img, (64, 64))
            test_image=np.expand_dims(test_image,axis=0)
            if a==1:
                plt.imshow(predictable_img,cmap='gray')

            result=cnn.predict(test_image)

            if result[0][0]==1:
                  prediction='HW'
            else:
                  prediction='DT'

            if prediction == 'HW':
                if( y < 0.2 * height_of_image or y > 0.8 * height_of_image or w > width_of_image/1.5):
                    continue
                
                else:
                    S=""
                    no_of_c+=1
                    #plt.imshow(predictable_img,cmap='gray')
                    boxes=cv2.rectangle(image, (x, y), (x+w, y+h), (36, 255, 12), 2)
                    predictable_img = cv2.cvtColor(predictable_img, cv2.COLOR_BGR2GRAY)
                    print(predictable_img.shape)
                    S=first1(predictable_img)
                    list_strings.append(S)

                    if no_of_c==3:
                        rd_hw_img=predictable_img
                        plt.imshow(rd_hw_img,cmap='gray')

    print('no of handwritten contors are ',no_of_c)
    st.image(boxes, caption='Bounding boxes image (OpenCV format)', use_column_width=True, channels='BGR')
    print('a is ',a)
    return list_strings

In [ ]:
if uploaded_file is not None:
    
    # Load the image into OpenCV format
    image_cv = load_image_into_cv2(uploaded_file)
    
    # Display the uploaded image using Streamlit
    st.image(image_cv, caption='Uploaded Image (OpenCV format)', use_column_width=True, channels='BGR')
    
    # Define the border to be removed (example values)
    top_border = 40
    bottom_border = 40
    left_border = 30
    right_border = 30
    
    # Crop the image using numpy slicing
    cropped_img = image_cv[top_border:image_cv.shape[0] - bottom_border, left_border:image_cv.shape[1] - right_border]
    
    #copying cropped image
    image=cropped_img.copy()
    base_image=image.copy()
    
    #grayscaling the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #blurring the image
    blur = cv2.GaussianBlur(gray, (7,7), 0)
    
    #thresholding the image
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    
    #kernel of image
    kernal = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 2))
    
    #dilate the image
    dilate = cv2.dilate(thresh, kernal, iterations=2)
    
    #finding the contours
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])
    
    list_strings_1=HWContours(cnts)
    print(list_strings_1)
    
    
    for l in range(len(list_strings_1)):
        prediction=list_strings_1[l]
        st.write(f"Prediction: {prediction}")
        